In [1]:
from src.sly_utils import download_project

In [2]:
import supervisely as sly
api = sly.Api()
project_id = 34717
info = api.project.get_info_by_id(project_id)
download_project(api, info, "app_data/smarttool_crops_tiny")

In [1]:
import mmengine
from src.dataset.make_infos import collect_mmdet3d_info
project_dir = "app_data/smarttool_crops_tiny"
mmdet3d_info = collect_mmdet3d_info(project_dir, "detection")
mmengine.dump(mmdet3d_info, f"{project_dir}/infos_train.pkl")
mmengine.dump(mmdet3d_info, f"{project_dir}/infos_val.pkl")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/local/lib/python3.8/dist-packages/mmdet3d/evaluation/functional/kitti_utils/eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_thresholds(scores: np.ndarray, num_gt, num_sample_pts=41):


In [5]:
mmdet3d_info["data_list"][0]

{'lidar_points': {'lidar_path': 'val/pointcloud/160.pcd', 'num_pts_feats': 6},
 'images': {},
 'instances': [{'bbox_3d': [38.8181037902832,
    20.23398780822754,
    -0.8008762001991273,
    1.47,
    4.05,
    1.53,
    -1.58],
   'bbox_label_3d': 0}],
 'cam_instances': None,
 'centerize_vector': array([-36.96401 , -19.701609,   0.      ], dtype=float32)}

In [10]:
import json
import os
import shutil
from mmdet3d.apis import Base3DInferencer

In [12]:
mmdetection3d_root_dir = Base3DInferencer._get_repo_or_mim_dir("mmdet3d")
model_list = list(reversed(json.load(open('model_list.json'))['detection_3d']))

## Copy configs

In [ ]:
dst_dir = "detection3d_configs"
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
os.makedirs(dst_dir)

# Copy base_configs to dst_dir/base
# Copy pre_trained_configs to dst_dir/pre_trained
os.makedirs(os.path.join(dst_dir, "base"))
os.makedirs(os.path.join(dst_dir, "pre_trained"))
for model in model_list:
    model_name = model['model_name']
    base_configs = model.get('base_configs', [])
    pre_trained_configs = model.get('pre_trained_configs', [])
    for config in base_configs:
        src = os.path.join(mmdetection3d_root_dir, config)
        dst = os.path.join(dst_dir, "base", os.path.basename(config))
        shutil.copy(src, dst)
    for pre_trained_config in pre_trained_configs:
        config = pre_trained_config['config']
        src = os.path.join(mmdetection3d_root_dir, config)
        dst = os.path.join(dst_dir, "pre_trained", os.path.basename(config))
        shutil.copy(src, dst)

In [27]:
from mmengine.config import Config, ConfigDict

# 1. Get model config
mmdetection3d_root_dir = "mmdetection3d"
model_list = list(reversed(json.load(open('model_list.json'))['detection_3d']))

model_item = model_list[0]
print(model_item)

model_name = model_item['model_name']
base_configs = model_item['base_configs']

is_pre_trained = False
config_path = f"{mmdetection3d_root_dir}/{base_configs[0]}"


# 2. Read parameters from config file
cfg = Config.fromfile(config_path)

{'name': 'CenterPoint', 'paper': 'CVPR', 'year': 2021, 'model_name': 'centerpoint', 'base_configs': ['configs/_base_/models/centerpoint_voxel01_second_secfpn_nus.py', 'configs/_base_/models/centerpoint_pillar02_second_secfpn_nus.py'], 'pre_trained_configs': [{'config': 'configs/centerpoint/centerpoint_voxel01_second_secfpn_head-circlenms_8xb4-cyclic-20e_nus-3d.py', 'weights': 'https://download.openmmlab.com/mmdetection3d/v1.0.0_models/centerpoint/centerpoint_01voxel_second_secfpn_circlenms_4x8_cyclic_20e_nus/centerpoint_01voxel_second_secfpn_circlenms_4x8_cyclic_20e_nus_20220810_030004-9061688e.pth', 'metadata': {'Training Memory (GB)': 5.2}, 'results': [{'Task': '3D Object Detection', 'Dataset': 'nuScenes', 'Metrics': {'mAP': 56.11, 'NDS': 64.61}}]}, {'config': 'configs/centerpoint/centerpoint_voxel01_second_secfpn_head-dcn-circlenms_8xb4-cyclic-20e_nus-3d.py', 'weights': 'https://download.openmmlab.com/mmdetection3d/v1.0.0_models/centerpoint/centerpoint_01voxel_second_secfpn_dcn_circ

In [58]:
from mmengine import Config
from src.config_factory.config_parameters import ConfigParameters
import re
p = ConfigParameters()
p.voxel_size = [0.55, 0.55, 0.15]

config_path = "/root/mmdetection3d-v1.x/detection3d_configs/base/centerpoint_pillar02_second_secfpn_nus.py"
cfg = Config.fromfile(config_path)
text = cfg.text


# substitute "voxel_size = [0.2, 0.2, 8]"
search_res = re.search("voxel_size\s*=\s*\[[0-9.]+,\s*[0-9.]+,\s*[0-9.]+\]", text)
if search_res:
    text = re.sub("voxel_size\s*=\s*\[[0-9.]+,\s*[0-9.]+,\s*[0-9.]+\]", f"voxel_size={p.voxel_size}", text, count=1)
else:
    print("voxel_size not found in config")


In [64]:
import os
# remove first string in text, as it is a path to config file
first_string = re.search("^.*\n", text).group(0)
if os.path.exists(first_string.strip()):
    text = re.sub("^.*\n", "", text, count=1)


In [69]:
cfg.get("point_cloud_range")

In [60]:
print(text)

/root/mmdetection3d-v1.x/detection3d_configs/base/centerpoint_pillar02_second_secfpn_nus.py
voxel_size=[0.55, 0.55, 0.15]
model = dict(
    type='CenterPoint',
    data_preprocessor=dict(
        type='Det3DDataPreprocessor',
        voxel=True,
        voxel_layer=dict(
            max_num_points=20,
            voxel_size=voxel_size,
            max_voxels=(30000, 40000))),
    pts_voxel_encoder=dict(
        type='PillarFeatureNet',
        in_channels=5,
        feat_channels=[64],
        with_distance=False,
        voxel_size=(0.2, 0.2, 8),
        norm_cfg=dict(type='BN1d', eps=1e-3, momentum=0.01),
        legacy=False),
    pts_middle_encoder=dict(
        type='PointPillarsScatter', in_channels=64, output_shape=(512, 512)),
    pts_backbone=dict(
        type='SECOND',
        in_channels=64,
        out_channels=[64, 128, 256],
        layer_nums=[3, 5, 5],
        layer_strides=[2, 2, 2],
        norm_cfg=dict(type='BN', eps=1e-3, momentum=0.01),
        conv_cfg=dict(type

In [29]:
re.sub("^.*\n", "", "asbb", count=1)

'asbb'

In [30]:
# remove first string in text using regex
text = re.sub("^.*\n", "", text, count=1)

# 3. cfg = Config.fromtext(text)
cfg = Config.fromstring(text, ".py")

# 4. (optional) update config recursively
# CenterPoint:

cfg.model.pts_voxel_encoder.voxel_size = p.voxel_size  # this was hardcoded in CenterPoint


In [47]:
import re

lidar_dims = 3
# substitute in_channels
search_res = re.search("in_channels\s*=\s*[0-6],", text)
if search_res:
    text = re.sub("in_channels\s*=\s*[0-6],", f"in_channels={lidar_dims},", text, count=1)
else:
    raise ValueError("in_channels not found in config")

In [48]:
re.findall("in_channels\s*=\s*[0-6]", text)

['in_channels=3',
 'in_channels=2',
 'in_channels=5',
 'in_channels=1',
 'in_channels=1',
 'in_channels=4']